In [36]:
import pandas as pd
import numpy as np
import datetime
from openpyxl import load_workbook

In [37]:
SEASON = 12
MATCHDAY = 7

In [38]:
mc = pd.read_excel('data/match_'+str(MATCHDAY)+'.xlsx')
mc.drop('№', axis=1, inplace=True)
mc.rename(columns={'index': 'Name'}, inplace=True)

In [39]:
mc.columns

Index(['Name', 'Итог', 'боруссия м. – байер', '0\n1',
       'хоффенхайм – вольфсбург', '2\n1', 'ростов – сочи', '0\n0',
       'астон вилла – вулверхэмптон', '0\n0.1', 'брайтон – лестер', '1\n2',
       'кёльн – вердер', '1\n1', 'атлетико – реал мадрид', '1\n1.1',
       'витесс – аз алкмаар', '2\n1.1', 'спартак – краснодар', '6\n1',
       'интер – аталанта', '1\n0'],
      dtype='object')

In [40]:
#Заголовки нулевой таблицы для создания ДБ ставок
zeros_col = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

matches = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='matches', index_col=0)
players = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='players', index_col=0)
season = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='season', index_col=0)
teams = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='teams', index_col=0)

In [41]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    return

def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] += total
    print(bets.iloc[player_id, 0], total)
    
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].iloc[match_ids] += 1
    return player_id

def get_match_ids(season=SEASON, matchday=MATCHDAY):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['Name', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

In [42]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
        print(f'{hometeam} - {awayteam} уже есть')
    else:
        matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                                  'H_score': int(h_score), 'A_score': int(a_score),
                                  'Season': SEASON, 'Matchday': MATCHDAY, 'Date': datetime.datetime.now(), 'Bets': 0,
                                  'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 
                                  'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids(SEASON,MATCHDAY)
bets = pd.DataFrame(pd.concat([mc['Name'], pd.DataFrame(np.zeros((mc['Name'].shape[0],8)), columns=zeros_col)], axis=1), columns=bets_columns)  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

<ipython-input-42-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-42-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-42-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-42-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-42-7dd6f4ee2aab>:5: UserWarning: Bool

In [43]:
for player in range(0,players['Name'].shape[0]):
    for match in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
        proceed_bet(player,match)

H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

Андреев Иван made no bet on боруссия м. - байер
Андреев Иван made no bet on хоффенхайм - вольфсбург
Андреев Иван made no bet on ростов - сочи
Зубатов Михаил made no bet on боруссия м. - байер
Зубатов Михаил made no bet on хоффенхайм - вольфсбург
Зубатов Михаил made no bet on ростов - сочи
Зубатов Михаил made no bet on астон вилла - вулверхэмптон
Зубатов Михаил made no bet on брайтон - лестер
Колодин Дмитрий made no bet on боруссия м. - байер
Колодин Дмитрий made no bet on хоффенхайм - вольфсбург
Колодин Дмитрий made no bet on ростов - сочи
Колодин Дмитрий made no bet on астон вилла - вулверхэмптон
Колодин Дмитрий made no bet on брайтон - лестер
Колодин Дмитрий made no bet on кёльн - вердер
Колодин Дмитрий made no bet on атлетико - реал мадрид
Колодин Дмитрий made no bet on витесс - аз алкмаар
Колодин Дмитрий made no bet on спартак - краснодар
Колодин Дмитрий made no bet on интер - аталанта


H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [44]:
mc

,Name,Итог,боруссия м. – байер,0\n1,хоффенхайм – вольфсбург,2\n1,ростов – сочи,0\n0,астон вилла – вулверхэмптон,0\n0.1,...,кёльн – вердер,1\n1,атлетико – реал мадрид,1\n1.1,витесс – аз алкмаар,2\n1.1,спартак – краснодар,6\n1,интер – аталанта,1\n0
0,Алферов Ян,7.0,2:1,0.0,1:2,0.0,2:3,0.0,1:0,0.0,...,0:1,0.0,1:0,0.0,1:2,0.0,2:1,2.0,2:0,2.0
1,Андреев Иван,22.0,NaN,0.0,NaN,0.0,NaN,0.0,2:0,0.0,...,1:1,5.0,1:1,5.0,2:1,5.0,2:1,2.0,3:1,2.0
2,Афанасьев Владимир,14.0,2:1,0.0,2:0,2.0,1:1,3.0,1:2,0.0,...,2:1,0.0,2:1,0.0,2:1,5.0,2:1,2.0,1:1,0.0
3,Белькин Сергей,9.0,2:0,0.0,1:2,0.0,1:0,0.0,2:1,0.0,...,0:1,0.0,2:0,0.0,2:1,5.0,2:2,0.0,3:1,2.0
4,Гришин Антон,24.0,2:1,0.0,1:2,0.0,1:1,3.0,1:1,3.0,...,1:1,5.0,1:1,5.0,1:1,0.0,1:1,0.0,2:1,3.0
5,Данилов Евгений,18.0,2:1,0.0,1:3,0.0,0:3,0.0,0:1,0.0,...,1:1,5.0,1:1,5.0,0:2,0.0,2:1,2.0,3:2,3.0
6,Еременко Владислав,18.0,1:1,0.0,2:1,6.0,0:1,0.0,1:1,3.0,...,0:1,0.0,2:1,0.0,0:1,0.0,1:0,2.0,2:0,2.0
7,Жигалов Александр,4.0,3:1,0.0,0:2,0.0,0:1,0.0,1:0,0.0,...,0:1,0.0,0:1,0.0,0:2,0.0,0:1,0.0,2:0,2.0
8,Зубатов Михаил,10.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,2:1,0.0,2:1,0.0,2:1,5.0,2:1,2.0,2:1,3.0
9,Колодин Дмитрий,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
bets['Name']

0             Алферов Ян
1           Андреев Иван
2     Афанасьев Владимир
3         Белькин Сергей
4           Гришин Антон
5        Данилов Евгений
6     Еременко Владислав
7      Жигалов Александр
8         Зубатов Михаил
9        Колодин Дмитрий
10        Конаков Никита
11       Котов Александр
12    Криеванс Владислав
13      Кучеренко Никита
14      Лесик Константин
15          Малевич Егор
16       Махмудов Руслан
17           Минеев Иван
18         Муратов Игорь
19        Невский Леонид
20     Новиков Александр
21        Оксанич Кирилл
22     Сибиряков Георгий
23       Титаренко Антон
24      Фёдоров Владимир
25     Чапаров Александр
26          Шевчук Антон
Name: Name, dtype: object

In [194]:
def find_match_id(tour):
    return 11+(tour-1)*3
find_match_id(9)

35

In [196]:
bets.iloc[6,35] = 7


In [45]:
for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Алферов Ян 7.0
Андреев Иван 22.0
Афанасьев Владимир 14.0
Белькин Сергей 9.0
Гришин Антон 24.0
Данилов Евгений 18.0
Еременко Владислав 18.0
Жигалов Александр 4.0
Зубатов Михаил 10.0
Колодин Дмитрий 0.0
Конаков Никита 10.0
Котов Александр 4.0
Криеванс Владислав 19.0
Кучеренко Никита 14.0
Лесик Константин 19.0
Малевич Егор 18.0
Махмудов Руслан 15.0
Минеев Иван 4.0
Муратов Игорь 11.0
Невский Леонид 18.0
Новиков Александр 19.0
Оксанич Кирилл 5.0
Сибиряков Георгий 16.0
Титаренко Антон 21.0
Фёдоров Владимир 13.0
Чапаров Александр 4.0
Шевчук Антон 15.0


<ipython-input-45-c524e9ee286a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
<ipython-input-41-0ecd95017322>:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Bombers

In [46]:
bombs = pd.read_excel('output/bombs_bets.xlsx', index_col=0, sheet_name=str(MATCHDAY))


In [47]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
2.0 Алферов Ян
2.0 Андреев Иван
6.0 Афанасьев Владимир
6.0 Белькин Сергей
2.0 Гришин Антон
2.0 Еременко Владислав
2.0 Жигалов Александр
2.0 Зубатов Михаил
2.0 Котов Александр
6.0 Криеванс Владислав
6.0 Кучеренко Никита
2.0 Лесик Константин
2.0 Малевич Егор
2.0 Махмудов Руслан
2.0 Муратов Игорь
2.0 Новиков Александр
2.0 Оксанич Кирилл
2.0 Сибиряков Георгий
2.0 Титаренко Антон
4.0 Фёдоров Владимир
6.0 Шевчук Антон


In [48]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [49]:
bets.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
4,Гришин Антон,26.0
12,Криеванс Владислав,25.0
1,Андреев Иван,24.0
23,Титаренко Антон,23.0
26,Шевчук Антон,21.0
20,Новиков Александр,21.0
14,Лесик Константин,21.0
15,Малевич Егор,20.0
13,Кучеренко Никита,20.0
2,Афанасьев Владимир,20.0


In [50]:
season.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
14,Лесик Константин,130.0
4,Гришин Антон,121.0
26,Шевчук Антон,119.0
21,Оксанич Кирилл,116.0
19,Невский Леонид,112.0
16,Махмудов Руслан,110.0
15,Малевич Егор,110.0
0,Алферов Ян,109.0
12,Криеванс Владислав,108.0
13,Кучеренко Никита,103.0


In [51]:
for match_id in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [52]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,леванте,NaN,1,0,1,0,25,1,5,0,19,1
1,гранада,NaN,1,0,1,0,25,1,5,0,19,1
2,майнц,NaN,1,1,0,0,27,1,2,1,23,1
3,унион берлин,NaN,3,1,1,1,76,10,5,1,60,1
4,ньюкасл юнайтед,NaN,1,1,0,0,27,0,4,3,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78,сочи,NaN,1,0,1,0,24,0,10,0,14,0
79,кёльн,NaN,1,0,1,0,26,9,0,0,17,0
80,витесс,NaN,1,1,0,0,26,4,2,0,20,0
81,аз алкмаар,NaN,1,0,0,1,26,4,2,0,20,0


In [53]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
book = load_workbook('output/bets.xlsx')
writer = pd.ExcelWriter('output/bets.xlsx', engine='openpyxl')
writer.book = book

# Write each dataframe to a different worksheet.
bets.to_excel(writer, sheet_name=str(MATCHDAY))

writer.save()
writer.close()

In [54]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'output/season_{str(SEASON)}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
season.to_excel(writer, sheet_name='season')
matches.to_excel(writer, sheet_name='matches')
players.to_excel(writer, sheet_name='players')
teams.to_excel(writer, sheet_name='teams')
#scorers.to_excel(writer, sheet_name='scorers_18')

writer.save()